In [ ]:
# --- Cell 0: Load Source Data ---
%matplotlib widget

import opym
from pathlib import Path
import importlib
import sys
import numpy as np
import matplotlib.pyplot as plt

import ipywidgets as widgets
from ipyfilechooser import FileChooser
from IPython.display import display, clear_output

# Reload modules
try:
    importlib.reload(opym.dataloader)
    importlib.reload(opym.petakit)
    importlib.reload(opym)
    print("Reloaded opym.dataloader and opym.petakit")
except Exception as e:
    print(f"Note: Could not reload all modules: {e}")


# --- Create Interactive Widgets ---
print("\n1. Select your test directory (containing one T/C stack).")
print("2. Click 'Load Data'.")

dir_chooser = FileChooser(
    path="/mmfs2/scratch/SDSMT.LOCAL/bscott/DataUpload",
    title="<b>Select Source TIFF Directory (e.g., '.../test_T0_C2'):</b>",
    show_only_dirs=True
)
load_button = widgets.Button(
    description="Load Data", button_style="primary"
)
load_output = widgets.Output()

# --- Global variables to hold data and parameters ---
source_processed_dir: Path | None = None
mip_original: np.ndarray | None = None
vmin_original: float = 0
vmax_original: float = 1

# Default physical params (from config.json)
default_xy_size = 0.136
default_z_step = 1.0
default_angle = 21.72


def on_load_button_clicked(b):
    global source_processed_dir, mip_original, vmin_original, vmax_original
    
    with load_output:
        clear_output(wait=True)
        if not dir_chooser.value:
            print("❌ ERROR: No directory selected.")
            return
        
        source_processed_dir = Path(dir_chooser.value)
        try:
            print(f"--- Loading source data from: {source_processed_dir.name} ---")

            get_stack, t, z, c, y, x = opym.load_tiff_series(source_processed_dir)
            
            if t > 0 or c > 0:
                print(f"⚠️ WARNING: This directory contains multiple T/C points.")
                print("   This tool will only process T=0, C=0.")
            
            # Load the one stack
            stack_original = get_stack(0, 0)
            
            # Calculate and store the original MIP
            mip_original = np.max(stack_original, axis=0)
            vmin_original, vmax_original = np.percentile(mip_original, [1, 99.9])
            if vmin_original >= vmax_original:
                vmax_original = np.max(mip_original)

            print(f"✅ Data loaded. Original MIP calculated.")
            print("\nYou can now run Cell 1 to tune parameters.")
            
        except FileNotFoundError as e:
            print(f"\n❌ ERROR: Could not load data.")
            print(f"  Details: {e}")
        except Exception as e:
            print(f"\n❌ An unexpected error occurred: {e}", file=sys.stderr)

load_button.on_click(on_load_button_clicked)
display(widgets.VBox([dir_chooser, load_button, load_output]))

In [ ]:
# --- Cell 1: Interactive Parameter Tuner ---

import ipywidgets as widgets
from IPython.display import display, clear_output
import numpy as np
import matplotlib.pyplot as plt

# --- Create Parameter Sliders ---
angle_slider = widgets.FloatSlider(
    value=default_angle,
    min=10.0,  # <-- Widened range
    max=40.0,  # <-- Widened range
    step=0.01,
    description='Sheet Angle:',
    readout_format='.2f',
    style={'description_width': 'initial'},
)
z_step_slider = widgets.FloatSlider(
    value=default_z_step,
    min=0.05,  # <-- Widened range
    max=5.0,   # <-- Widened range
    step=0.05,
    description='Z-Step (um):',
    readout_format='.2f',
    style={'description_width': 'initial'},
)
xy_size_slider = widgets.FloatSlider(
    value=default_xy_size,
    min=0.05,
    max=0.2,
    step=0.001,
    description='XY Pixel Size (um):',
    readout_format='.3f',
    style={'description_width': 'initial'},
)

# --- Create Buttons and Output Areas ---
run_view_button = widgets.Button(
    description="Run & View Processed Stack", button_style="success"
)
log_output = widgets.Output(layout={'border': '1px solid black', 'height': '200px', 'overflow_y': 'scroll'})
view_output = widgets.Output() # This will hold the plot

# --- Define Button Click Handler ---
def on_run_view_button_clicked(b):
    run_view_button.disabled = True
    
    with log_output:
        clear_output(wait=True)
    with view_output:
        clear_output(wait=True) # Clear previous plot

    try:
        if source_processed_dir is None or mip_original is None:
            with log_output:
                print("❌ ERROR: Source data not loaded. Please run Cell 0 first.")
            return

        # 1. Get all parameters from widgets
        angle = angle_slider.value
        z_step = z_step_slider.value
        xy_size = xy_size_slider.value
        
        with log_output:
            print(f"Processing with Angle={angle}, Z-Step={z_step}, XY-Size={xy_size}...")

        # 2. Create unique directory name for this run
        dsr_dir_name = f"DSR_a{angle:.2f}_z{z_step:.2f}_xy{xy_size:.3f}"
        ds_dir_name = f"DS_a{angle:.2f}_z{z_step:.2f}_xy{xy_size:.3f}"
        
        # 3. Run the standard processing function
        opym.run_petakit_processing(
            source_processed_dir,
            ds_dir_name=ds_dir_name,
            dsr_dir_name=dsr_dir_name,
            sheet_angle_deg=angle,
            z_step_um=z_step,
            xy_pixel_size=xy_size,
            save_mip=True,  # Might as well save the MIPs
        )
        
        # 4. Load the processed stack
        output_path = source_processed_dir / dsr_dir_name
        with log_output:
            print(f"✅ Processing complete. Loading results from: {output_path.name}")

        get_stack_proc, t, z, c, y, x = opym.load_tiff_series(output_path)
        stack_processed = get_stack_proc(0, 0)
        
        # 5. Calculate the new MIP
        mip_processed = np.max(stack_processed, axis=0)
        vmin_proc, vmax_proc = np.percentile(mip_processed, [1, 99.9])
        if vmin_proc >= vmax_proc:
            vmax_proc = np.max(mip_processed)

        # 6. --- Display the results ---
        with view_output:
            fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))
            
            ax1.imshow(mip_original, cmap='gray', vmin=vmin_original, vmax=vmax_original)
            ax1.set_title(f"Original MIP (Shape: {mip_original.shape})")
            ax1.axis('off')
            
            ax2.imshow(mip_processed, cmap='gray', vmin=vmin_proc, vmax=vmax_proc)
            ax2.set_title(f"Processed MIP (Shape: {mip_processed.shape})")
            ax2.axis('off')
            
            plt.tight_layout()
            plt.show()

    except Exception as e:
        with log_output:
            print(f"\n❌ An unexpected error occurred: {e}", file=sys.stderr)
            traceback.print_exc(file=sys.stderr)
    finally:
        # Re-enable the button
        run_view_button.disabled = False


# --- Link button and display UI ---
run_view_button.on_click(on_run_view_button_clicked)

param_box = widgets.VBox([
    angle_slider,
    z_step_slider,
    xy_size_slider
])

display(widgets.VBox([param_box, run_view_button, log_output, view_output]))